In [20]:
%%time

import numpy as np
from numpy import percentile
import pandas as pd
import os
import copy
import time

!pip3 install pickle5
import pickle5 as pickle

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
np.random.seed(22)

import tensorflow as tf
#tf.debugging.set_log_device_placement(True)


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model
import pydot
#import tensorflow_docs as tfdocs

print(tf.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
!git clone https://github.com/Juniper82/SIDIS-Affinity

%cd '/content/SIDIS-Affinity'

# google hosted test data
collinear_feild = '1M2MprjFCMT0DfqX7kV_dF591YZGJAhBl' # collinear
current_feild = '1WwNcRixgruw784SCLf9iP4rM0p6V3HhM' # current
target_feild = '1nrd9cLrDUK4w_4upGvWz2qe5l7hrNXqR' # target
tmd_feild = '1Iz1REawJ7W8s3xmAG-dykeg0isPLR8UA' # tmd

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$collinear_feild' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$collinear_feild" -O collinear && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$current_feild' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$current_feild" -O current && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$target_feild' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$target_feild" -O target && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$tmd_feild' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$tmd_feild" -O TMD && rm -rf /tmp/cookies.txt


2.7.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5670746506294503159
xla_global_id: -1
]
fatal: destination path 'SIDIS-Affinity' already exists and is not an empty directory.
/content/SIDIS-Affinity
--2021-11-25 06:11:01--  https://drive.google.com/file/d/1M2MprjFCMT0DfqX7kV_dF591YZGJAhBl/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 173.194.218.102, 173.194.218.138, 173.194.218.113, ...
Connecting to drive.google.com (drive.google.com)|173.194.218.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘EIC_aff_test_collinearaff.csv’

EIC_aff_test_collin     [ <=>                ]  65.06K  --.-KB/s    in 0.004s  

2021-11-25 06:11:01 (18.0 MB/s) - ‘EIC_aff_test_collinearaff.csv’ saved [66625]

--2021-11-25 06:11:02--  https://drive.google.com/file/d/1WwNcRixgruw784SCLf9iP4rM0p6V3HhM/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 172.217.204.

In [21]:
# Define plotting function


def plotPred(trueVal,predVal,title,threshold):
    fig=plt.figure(figsize=(10,10))
    a = plt.axes(aspect='equal')
    plt.scatter(trueVal, predVal,alpha=0.05)
    plt.xlabel('True Values [Affinity]',fontsize=20)
    plt.ylabel('Predictions [Affinity]',fontsize=20)
    plt.title(title,fontsize=25)
    lims = [0, 1]
    plt.xlim(lims)
    plt.ylim(lims)
    _ = plt.plot(lims, lims ,color = 'k',linestyle='-')
    # add lines for residual threshold and center
    x1 = np.linspace(0,1-threshold,100)
    plt.plot(x1,x1+threshold, color = 'r',linestyle='--')
    x2 = np.linspace(threshold,1,100)
    plt.plot(x2,x2-threshold, color = 'r',linestyle='--')
    plt.show()
#     from matplotlib.backends.backend_pdf import PdfPages
#     with PdfPages('../figures/tmd_sigmoid.pdf') as pdf:
#         pdf.savefig(fig)
#    plt.savefig('../figures/tmd_sigmoid.png')


def getThresh(residuals):
    badRes = np.ones(len(residuals)).tolist()
    lowThresh = 0.0000000001
    upperThresh = 1.
    threshold = lowThresh
    tolerance = 0.000001
    atol = len(badRes)/len(residuals)
    iterCount = 0

    while np.abs(atol - .05) > tolerance :

        badRes = [res for res in residuals if np.abs(res) > threshold]
        atol = len(badRes)/len(residuals)

        if np.abs(atol - 0.05) < tolerance: 
            break

        else:
            if atol < 0.05:
                upperThresh = threshold
            else:
                lowThresh = threshold
                
        threshold = (upperThresh + lowThresh)/2 
        iterCount += 1

    print('# residuals: ',len(residuals))
    print('# residuals outside threshold: ',len(badRes))
    print('percent residuals outside threshold: ',len(badRes)/len(residuals))
    print('max badRes: ', max(np.abs(badRes)),', max residual: ',max(np.abs(residuals)))
    print('min BadRes: ', min(np.abs(badRes)))
    print('threshold: ', threshold)
    print('# of iterations: ',iterCount)
    print('#'*12,'\n'*2)
    print('Percent residuals within +/- %s' % threshold)
    accuracy = len([res for res in residuals if np.abs(res) <= threshold])/len(residuals)
    print('accuracy: ',accuracy)
    print('Percent residuals == 0.0')
    accuracy = len([res for res in residuals if np.abs(res) == 0.0])/len(residuals)
    print('accuracy: ',accuracy)
    return(threshold)


def getThresh2(residuals):
    threshold = percentile(residuals, [2.5, 97.5])
    print('Percent residuals within %s' % threshold)
    accuracy = np.mean((residuals > threshold[0]) & (residuals < threshold[1]))
    print('accuracy: ',accuracy)
    print('Percent residuals == 0.0')
    accuracy = np.mean(residuals == 0.0)
    print('accuracy: ',accuracy)
    return(threshold)


def fivenumsum(resids):
    
    # calculate a 5-number summary
    from numpy import percentile
    quartiles = percentile(resids, [0, 25, 50, 75, 100])
    # print 5-number summary
    print('Min: %.3f' % quartiles[0])
    print('Q1: %.3f' % quartiles[1])
    print('Median: %.3f' % quartiles[2])
    print('Q3: %.3f' % quartiles[3])
    print('Max: %.3f' % quartiles[4])
    


In [22]:
# # test tmd model and create plots for visualization of results
# model_name = 'tmd_DNN_V2_sigmoid_checkpoint_model'

# model_name=name+'_cp_model'

def scot_plot(mld_path, file_path, lable_name, ):
    
    test_results = {}
    model = tf.keras.models.load_model(mld_path)
    # test_features = np.load(file_path,allow_pickle=True)
    test_features = pd.read_csv(file_path)
    
    # test_features = np.array(test_features, np.float)
    #test_features = test_features.drop(columns=['currentaff','targetaff', 'collinearaff','hadron'])
    #print(test_features.head())
    test_labels = test_features.pop(lable_name)
    print('Model Configureation')
    print('#'*12,'\n'*2)
    for i, layer in enumerate (model.layers):
        print ('layer', i, layer)
        try:
            print ("    ", layer.activation)
        except AttributeError:
            print('   no activation attribute')
        
    print('#'*12,'\n'*2)
    print('Model Summary')
    print(model.summary())
    test_results[model_name] = model.evaluate(test_features, test_labels, verbose=0)
    test_predictions = model.predict(test_features).flatten()
    print('#'*12,'\n'*2)
#     negPred=[value for value in test_predictions if value < 0]
#     giantPred=[value for value in test_predictions if value > 1]
#     print('# of predictions < 0:',len(negPred),'\n# of predictions > 1:',len(giantPred))

#     if len(negPred)==0 and len(giantPred)==0:
#         print('no correction needed')
#     else:
#         print('setting negative predictions to 0\nsetting predictions > 1 to 1')
#         test_predictions[test_predictions<0] = 0
#         test_predictions[test_predictions>1] = 1
#         print('testing affinity correction...')
#         negPred=[value for value in test_predictions if value < 0]
#         giantPred=[value for value in test_predictions if value > 1]
#         print('# of predictions < 0:',len(negPred),'\n# of predictions > 1:',len(giantPred))

    print('#'*12,'\n'*2)
    residuals = test_labels-test_predictions
    threshold = getThresh(residuals)
    _ = getThresh2(residuals)
    print('#'*12,'\n'*2)
    # badRes = [res for res in list(residuals) if np.abs(res) >= threshold] # residuals that are above threshold
    # #print(badRes)
    # percentBadRes = len(badRes)/len(residuals)

    # Calculate R squared
    print('----------------------------')
    print('\nModel Stats')
    R2 = r2_score(test_labels, test_predictions, multioutput='variance_weighted')
    print('mse: ',test_results[model_name])
    print('R-squared: ',R2)
    # print(np.round(percentBadRes,6),'percent of residuals were above threshold 0.2 for',len(test_labels),' predictions')
    # print(len(badRes),' residuals were above threshold 0.2 for',len(test_labels),' predictions')
    # print('max |residual|: ',max(np.abs(residuals)))
    print('#'*12,'\n'*2)
    print('Five number summary of residuals')
    fivenumsum(residuals)
    print('#'*12,'\n'*2)
    plotPred(test_labels,test_predictions,model_name+' sigmod model',threshold)
    plt.figure(figsize=(10,10))
    plt.hist(residuals,bins=100)
    plt.show()



# TMD 

In [37]:
%%time

# region_name = 'collinear'
# region_name = 'target'
# region_name = 'current'
region_name = 'TMD'
lable_name = region_name + 'aff'
file_name = region_name
file_path = './' + file_name
model_name = 'final_' + region_name
mld_path = './models/' + model_name

scot_plot(mld_path, file_path, lable_name)

KeyError: ignored

# Current

In [ ]:
%%time

# region_name = 'collinear'
# region_name = 'target'
region_name = 'current'
# region_name = 'TMD'
lable_name = region_name + 'aff'
file_name = region_name
file_path = './' + file_name
model_name = 'final_' + region_name
mld_path = './models/' + model_name

scot_plot(mld_path, file_path, lable_name)

# Collinear

In [ ]:
%%time

region_name = 'collinear'
# region_name = 'target'
# region_name = 'current'
# region_name = 'TMD'
lable_name = region_name + 'aff'
file_name = region_name
file_path = './' + file_name
model_name = 'final_' + region_name
mld_path = './models/' + model_name

scot_plot(mld_path, file_path, lable_name)

# target

In [ ]:
%%time

# region_name = 'collinear'
region_name = 'target'
# region_name = 'current'
# region_name = 'TMD'
lable_name = region_name + 'aff'
file_name = region_name
file_path = './' + file_name
model_name = 'final_' + region_name
mld_path = './models/' + model_name

scot_plot(mld_path, file_path, lable_name)